In [1]:
import pandas as pd
import numpy as np

In [2]:
age_test_data = pd.read_csv('../data/age_test.csv',names=['uid'])
age_train_data = pd.read_csv('../data/age_train.csv',names=['uid','age_group'])

In [3]:
app_feature = pd.read_csv('../feature/app_feature.csv')

In [4]:
most_dura = pd.read_csv('../feature/use_most_eight_duration_app.csv')

In [5]:
user_app_actived = pd.read_csv('../data/user_app_actived.csv',names=['uid','appid_2'])

In [6]:
user_app_actived['len'] = user_app_actived['appid_2'].apply(lambda x:len(x))
print(len(user_app_actived))
user_app_actived = user_app_actived[user_app_actived['len']>5]
user_app_actived.drop(columns=['len'],inplace=True)
print(len(user_app_actived))

2512500
2508762


In [7]:
most_dura = most_dura[['uid','appid']]

In [8]:
age_train_data = age_train_data.merge(most_dura,on=['uid'],how='left')

In [9]:
age_train_data = age_train_data.merge(user_app_actived,on=['uid'],how='left')

In [10]:
age_test_data = age_test_data.merge(most_dura,on=['uid'],how='left')
age_test_data = age_test_data.merge(user_app_actived,on=['uid'],how='left')

In [11]:
######app对应的各个年龄段的数量。。。可能导致数据穿越,过拟合
app_age = {}
app_list = age_train_data['appid'].values
app_age_list = age_train_data['age_group'].values
for num,i in enumerate(app_age_list):
    app = app_list[num]
    if app is np.nan:
        continue
    if i in app_age:
        for j in app.split('#'):
            if j in app_age[i]:
                app_age[i][j] = app_age[i][j]+1
            else:
                app_age[i][j] = 1
    else:
        for j in app.split('#'):
            app_age[i] = {}
            app_age[i][j] = 1

In [12]:
app_age_value = pd.DataFrame(app_age)

In [13]:
app_age_value = app_age_value.reset_index().rename(columns={'index':'appid'})

In [14]:
age_train_data['appid'] = age_train_data.apply(lambda x:x['appid'] if x['appid'] is not np.nan else x['appid_2'],axis=1)
age_test_data['appid'] = age_test_data.apply(lambda x:x['appid'] if x['appid'] is not np.nan else x['appid_2'],axis=1)  

In [15]:
age_train_data.drop(columns=['appid_2'],inplace=True)
age_test_data.drop(columns=['appid_2'],inplace=True)

In [16]:
age_train_data['appid'] = age_train_data['appid'].fillna('a00289791#a0048467')
age_test_data['appid'] = age_test_data['appid'].fillna('a00289791#a0048467')

In [17]:
age_train_data['appid'].isnull().sum()

0

In [18]:
app_feature = pd.read_csv('../feature/app_feature.csv')

In [19]:
app_feature = app_feature.merge(app_age_value,on='appid',how='left')

In [21]:
app_feature['allnum'] = app_feature[1]+app_feature[2]+app_feature[3]+app_feature[4]+app_feature[5]+app_feature[6]
for i in [1,2,3,4,5,6]:
    app_feature[i] = app_feature[i]/(app_feature['allnum']+1)

In [22]:
for i in [1,2,3,4,5,6]:
    app_feature[i] = app_feature[i].fillna(app_feature[i].mean())

In [23]:
app_feature['app_cate'] = app_feature['app_cate'].map(dict(zip(app_feature['app_cate'].unique(), range(0, app_feature['app_cate'].nunique()))))
for i in app_feature.columns:
    if i=='appid':
        continue
    app_feature[i] = app_feature[i].fillna(app_feature[i].mean())

In [24]:
app_dict = dict(zip(app_feature.values[:,0],app_feature.values[:,1:]))

In [25]:
def get_app_feature_vec(a):
    featurevec = np.zeros([15,39])
    for num,i in enumerate(a.split('#')):
        if num>=15:
            break
        featurevec[num,:] = app_dict[i]
    return featurevec
age_train_data['appid'] = age_train_data['appid'].apply(lambda x:get_app_feature_vec(x))

In [26]:
age_test_data['appid'] = age_test_data['appid'].apply(lambda x:get_app_feature_vec(x))

In [27]:
train_fea = np.array(age_train_data['appid'].tolist())
test_fea = np.array(age_test_data['appid'].tolist())

In [28]:
np.save('../feature/nn_train.npy',train_fea)
np.save('../feature/nn_test.npy',test_fea)